In [ ]:
!pip install -r requirements.txt

In [ ]:
DATA_PATH = "Scraped_Car_Review_dodge.csv"

import time
from tqdm import tqdm
import polars as pl
import torch
from transformers import (
    pipeline,
    TextClassificationPipeline,
)

In [ ]:
reviews_list = pl.read_csv(DATA_PATH)["Review"].to_list()
len(reviews_list)

In [ ]:
reviews_list[0]

In [ ]:
def time_text_classifier(
    text_pipeline: TextClassificationPipeline,
    texts: list[str],
    batch_size: int = 1,
) -> None:
    """Time how long it takes a TextClassificationPipeline
    to run inference on a list of texts"""

    texts_generator = (t for t in texts)
    pipeline_iterable = tqdm(
        text_pipeline(
            texts_generator,
            batch_size=batch_size,
            truncation=True,
            max_length=500,
        ),
        total=len(texts),
    )

    for result in pipeline_iterable:
        pass

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_pipeline_cpu = pipeline(model=model_name, device=-1)
sentiment_pipeline_gpu = pipeline(model=model_name, device=0)

In [ ]:
time_text_classifier(sentiment_pipeline_cpu, reviews_list[0:1000])

In [ ]:
time_text_classifier(sentiment_pipeline_gpu, reviews_list[0:1000])

In [ ]:
batch_sizes = [1, 2, 4, 8, 10, 12, 15, 20, 50, 100]

for batch_size in batch_sizes:
    print(f"Batch size: {batch_size}")
    time_text_classifier(
        sentiment_pipeline_gpu, reviews_list, batch_size=batch_size
    )